[View in Colaboratory](https://colab.research.google.com/github/agerk/dataScience/blob/master/text_mining_medical_data_python.ipynb)

<h2>Text Mining: Extracting dates from medical data</h2>

## Data discription

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

objective: With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.




In [0]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [0]:
# cardinality
ZERO_OR_MORE = '*'
ONE_OR_MORE = '+'
ZERO_OR_ONE = '?'
EXACTLY_TWO = '{2}'
ONE_OR_TWO = '{1,2}'
EXACTLY_ONE = '{1}'

# Groups and Classes
GROUP = r"({})"
NAMED = r"(?P<{}>{})"
CLASS = '[{}]'
NEGATIVE_LOOKAHEAD = '(?!{})'
NEGATIVE_LOOKBEHINED = '(?<!{})'
POSITIVE_LOOKAHEAD = '(?={})'
POSITIVE_LOOKBEHINED = '(?<={})'
ESCAPE = '\{}'

# Numbers
DIGIT = r"\d"
ONE_DIGIT = DIGIT + EXACTLY_ONE
ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO 
NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
TWO_DIGITS = DIGIT + EXACTLY_TWO
THREE_DIGITS = DIGIT + '{3}'
EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
FOUR_DIGITS = DIGIT + r"{4}" + NON_DIGIT

# String literals
SLASH_FORWARD = r"/"
OR = r'|'
LOWER_CASE = 'a-z'
SPACE = "\s"
DOT = '.'
DASH = '-'
COMMA = ','
PUNCTUATION = CLASS.format(DOT + COMMA + DASH)  # combine punctuation together
EMPTY_STRING = ""

# Dates
MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
               + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
MONTHS = [x + MONTH_SUFFIX for x in MONTH_PREFIXES]
MONTHS = GROUP.format(OR.join(MONTHS))
DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
YEAR = FOUR_DIGITS

**Helper functions**

In [0]:
# filter dates start with string
CONTRACTED = (ONE_OR_TWO_DIGITS + LOWER_CASE + EXACTLY_TWO)
CONTRACTION = NAMED.format("contraction", MONTHS + CONTRACTED
                           + DAY_SUFFIX + YEAR)

# get dates that have no days-- Mar 21st, 2009
NO_DAY_BEHIND = NEGATIVE_LOOKBEHINED.format(DIGIT + SPACE)
NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)

# most common date format -- May 20, 2018
WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)
# cases where the day is placed before the mont -- 20 March 2000
BACKWARDS = NAMED.format('backwards', ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)

# cases where slashes are used with only two digits-- 8/4/10
SLASHED = SLASH_FORWARD.join([ONE_OR_TWO_DIGITS,
                             ONE_OR_TWO_DIGITS,
                             EXACTLY_TWO_DIGITS])

DASHED = DASH.join([ONE_OR_TWO_DIGITS,
                   ONE_OR_TWO_DIGITS,
                   EXACTLY_TWO_DIGITS])

TWENTIETH_CENTURY = NAMED.format("twentieth",
                                OR.join([SLASHED, DASHED]))

# case where digits with slashes are used and 4 digits are used for the year-- 7/5/2998
NUMERIC = NAMED.format("numeric", SLASH_FORWARD.join([ONE_OR_TWO_DIGITS,
                                               ONE_OR_TWO_DIGITS,
                                               FOUR_DIGITS]))

# for cases where only month and year are given -- 8/2011
NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHINED.format(SLASH_FORWARD)
NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHINED.format(CLASS.format(SLASH_FORWARD + DIGIT))

NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
             + ONE_DIGIT
             + SLASH_FORWARD
             + FOUR_DIGITS)

NO_TWO_DAYS = (NO_PRECEDING_SLASH
              + TWO_DIGITS
              + SLASH_FORWARD
              + FOUR_DIGITS)

NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                             NO_ONE_DAY + OR + NO_TWO_DAYS)

In [0]:
# For cases where only a year is given
CENTURY = GROUP.format('19' + OR + '20') + TWO_DIGITS
DIGIT_SLASH = DIGIT + SLASH_FORWARD
DIGIT_DASH = DIGIT + DASH
DIGIT_SPACE = DIGIT + SPACE
LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
COMMA_SPACE = COMMA + SPACE
YEAR_PREFIX = NEGATIVE_LOOKBEHINED.format(OR.join([DIGIT_SLASH, DIGIT_DASH,
                                                  DIGIT_SPACE, LETTER_SPACE, 
                                                  COMMA_SPACE]))

YEAR_ONLY = NAMED.format('year_only', YEAR_PREFIX + CENTURY)

In [0]:
# Get leftovers that dont match any of the above
IN_PREFIX = POSITIVE_LOOKBEHINED.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
SINCE_PREFIX = POSITIVE_LOOKBEHINED.format(CLASS.format('Ss') + 'ince' + SPACE) + CENTURY
AGE = POSITIVE_LOOKBEHINED.format('Age' + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
AGE_COMMA = POSITIVE_LOOKBEHINED.format('Age' + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
OTHERS = ['delivery', 'quit', 'attempt', 'nephrectomy', THREE_DIGITS]
OTHERS = [POSITIVE_LOOKBEHINED.format(label + SPACE) + CENTURY for label in OTHERS]
OTHERS = OR.join(OTHERS)
LEFTOVERS_PREFIX = OR.join([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
LEFTOVERS = NAMED.format('leftovers', LEFTOVERS_PREFIX)

In [0]:
# SINCE_PREFIX

In [0]:
def combine_date():
    '''
    Args: None
    Returns: output re expression to handle combining all possible date format  
    '''
    DATE = NAMED.format('date', OR.join([NUMERIC,TWENTIETH_CENTURY,
                                        WORDS, BACKWARDS, CONTRACTION,
                                        NO_DAY, NO_DAY_NUMERIC,
                                        YEAR_ONLY, LEFTOVERS]))
    return DATE

In [0]:
# DATE = NAMED.format('date', OR.join([NUMERIC,TWENTIETH_CENTURY,
#                                         WORDS, BACKWARDS, CONTRACTION,
#                                         NO_DAY, NO_DAY_NUMERIC,
#                                         YEAR_ONLY, LEFTOVERS]))

In [0]:
def twentieth_century(date):
    '''
    Args: date
    Returns:  adds a 19 to the year
    '''
    month, day, year = date.group(1).split(SLASH_FORWARD)
    year = '19{}'.format(year)
    return SLASH_FORWARD.join([month, day, year])


In [0]:
def take_two(line):
    match = re.search(TWENTIETH_CENTURY, line)
    if match:
        return twentieth_century(match)
    return line

In [0]:
# apply grammer
def extract_and_count(expression, data, name):
    '''
    Args: regular expression to match input data
    Returns: extract all matches and report the counts. tuple(date, int)
    '''
    extracted = data.str.extractall(expression)[name]
    count = len(extracted)
    return extracted , count

In [0]:
found = df.str.extractall(combine_date())
date = combine_date()
total_year_found = len(found.date)
# total_year_found
# check missing value
missing_data = [label for label in df.index if label not in found.index.levels[0]]
try:
    print(missing_data[0], df.loc[missing_data[0]])
except IndexError:
    print("all rows matched")

all rows matched


In [1]:
# Clean data

def clean_data(source, expression, replacement, sample=5):
    """applies the replacement to the source
    Args: source of string,
        expression (str): re to match what to replace
        replacement: function/ or expression to replace the matching expression
        samplele: randomly chosen examples for checking
    Returns: cleaned version of source
    """
#     print("Random sample before clean-up: {}".format(source.sample(sample)))
    cleaned_df = source.str.replace(expression, replacement)
#     print("\nRandom sample after clean-up: {}".format(cleaned_df.sample(sample)))
    # check length
    assert len(source) == len(cleaned_df)
    return cleaned_df

# clean punctuation
def clean_punc(source, sample=5):
    """
    Args: data to clean punctuation
    Returns: cleaned data with puctuation removed
    """
    if any(source.str.contains(PUNCTUATION)):
        source = clean_data(source, PUNCTUATION, EMPTY_STRING)
    
    return source

# convert long month names to three-letter form

LONG_MONTH_TO_SHORT = dict(January="Jan", February="Feb", March="Mar",
                         April="Apr", May="May", June="Jun",
                         July="Jul", August="Aug", September="Sep",
                         October="Oct", November="Nov", December="Dec")

# fuzzy match for words with spelling erros
LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                   + CLASS.format(LOWER_CASE)
                                   + ZERO_OR_MORE
                                   for month in LONG_MONTH_TO_SHORT.values()])

# 
def long_month_to_short(match):
    """
    Args: object mathcing a long month
    Returns: shortned version of the month
    """
    return match.group(match.lastindex)

# conver long month names to short
def convert_long_months_to_short(source, sample=5):
    """
    Args: source data with months
    Returns: converted data with short months
    """
    short_month = clean_data(source, LONG_TO_SHORT_EXPRESSION, long_month_to_short)
    
    return short_month

# add January 1 to year only dates
def add_month_date(match):
    """
    Args: object that mathces a 4-digit year
    Returns: string with "01/01/
    """
    return "01/01/" + match.group()

# add Juan 1 to df
def add_january_one(source):
    """ adds 01/01 to year-only dates
    
    Args: data with the date
    Returns: year in source with 01/01 prefixed
    """
    return clean_data(source, YEAR_ONLY, add_month_date)

# Two digits number
# make sure that there are only two digits number

two_digits_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH_FORWARD)

def two_digits(match):
    """ add a leading zero if necessary
    
    Args: date with one or two digits
    Returns: matched string with leading zero if needed
    """
    return "{:02}".format(int(match.group()))

# clean when two digits are followed by a slash

def clean_two_digits(source, sample=5):
    """ make sure data has two-digits
    
    Args: data with digits followed by slash and sample
    Returns: data with digits coerced to two digits
    """
    
    return clean_data(source, two_digits_expression, two_digits, sample)

# clean two_digits and check for trainling slas
def clean_two_digits_isolated(source, sample=5):
    """ cleans two digits that are standalone
    
    Args: 
        source of the data
        sample to show
    
    Returns:
        converted data
    """
    return clean_data(source, ONE_OR_TWO_DIGITS, two_digits, sample)

# clean-up monnths
digits = ("{:02}".format(month) for month in range(1, 13))
MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))

def month_to_digits(match):
    """ converts short mont to digits
    
    Args:
        match object with short-month
    Returns:
        str-- month as two-digits number 
    
    """
    return MONTH_TO_DIGITS[match.group()]

# convert short month to digits
def convert_short_month_to_digits(source, sample=5):
    """ convert three-letter month to two-dogots
    
    Args:
        data with three-letter months
        sample
    Returns:
        source with short-months converted to digits
    
    """
    return clean_data(source, SHORT_MONTHS_EXPRESSION, month_to_digits, sample)

# clean months by combining all cases
def clean_months(source, sample=5):
    """ clean up months
    
    Args: source of the months
    Returns: celaned month
    """
    cleaned = clean_punc(source)
    cleaned = clean_data(cleaned, LONG_TO_SHORT_EXPRESSION,
                        long_month_to_short, sample)
    cleaned = clean_data(cleaned, SHORT_MONTHS_EXPRESSION, 
                        month_to_digits, sample)
    return cleaned

# Frame to series: for cases where the date-fields were broken
def frame_to_series(frame, index_source, sample=5):
    """ re-combine data-frame into a series
    
    Args:
        frame with month, day, year columns
        index_source: source to copy from
        sample
        
    Returns:
        panads.Series with dates as month/day/year
    """
    combined = frame.month + SLASH_FORWARD + frame.day + SLASH_FORWARD + frame.year
    combined.index = index_source.index
    
    return combined 

# get cleaned year only data
year_only, year_only_count = extract_and_count(YEAR_ONLY, df, "year_only")
year_only_cleaned = add_january_one(year_only)

# leftovers
leftovers, leftovers_count = extract_and_count(LEFTOVERS, df, "leftovers")
leftovers_cleaned = add_january_one(leftovers)

cleaned = pd.concat([year_only_cleaned, leftovers_cleaned])
# print(len(cleaned))

# No day numerics
no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, df,
                                                         "no_day_numeric")
no_day_numeric_cleaned = clean_two_digits(no_day_numeric)
no_day_numeric_cleaned = clean_data(no_day_numeric_cleaned, SLASH_FORWARD, lambda m: "/01/")

# add no_numeric dat to cleaned df
original = len(cleaned)
cleaned = pd.concat([cleaned, no_day_numeric_cleaned])
assert len(cleaned) == no_day_numeric_count + original

# no day
no_day, no_day_count = extract_and_count(NO_DAY, df, 'no_day')
no_day_cleaned = clean_months(no_day)
no_day_cleaned= clean_data(no_day_cleaned, SPACE + ONE_OR_MORE,
                          lambda match: "/01/")
original = len(cleaned)
cleaned = pd.concat([cleaned, no_day_cleaned])
assert len(cleaned) == no_day_count + original

# Backwards
backwards, backwards_count = extract_and_count(BACKWARDS, df, 'backwards')
frame = pd.DataFrame(backwards.str.split().tolist(),
                    columns = "day month year".split())
frame.day = clean_two_digits(frame.day)
frame.month = clean_months(frame.month)

# combine frames together

backwards_cleaned = frame_to_series(frame, backwards)

original = len(cleaned)
cleaned = pd.concat([cleaned, backwards_cleaned])
assert len(cleaned) == backwards_count + original

# Words
words, words_count = extract_and_count(WORDS, df, 'words')
frame = pd.DataFrame(words.str.split().tolist(),
                    columns="month day year".split())
# print(frame.head())
frame.month = clean_months(frame.month)
frame.day = clean_punc(frame.day)

# word cleaned
words_cleaned = frame_to_series(frame, words)
# print(frame.head())
# check 
original = len(cleaned)
cleaned = pd.concat([cleaned, words_cleaned])
assert len(cleaned) == words_count + original

# Twentieth century
twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, df, 'twentieth')
twentieth_century_cleaned = twentieth.str.replace(DASH, SLASH_FORWARD)

frame = pd.DataFrame(twentieth_century_cleaned.str.split(SLASH_FORWARD).tolist(),
                    columns=["month", "day", "year"])

# Month
frame.month = clean_two_digits_isolated(frame.month)
frame.day = clean_two_digits_isolated(frame.day)
frame.year = clean_data(frame.year, TWO_DIGITS, lambda match: "19" + match.group())
twentieth_century_cleaned = frame_to_series(frame, twentieth)

original = len(cleaned)
cleaned = pd.concat([cleaned, twentieth_century_cleaned])
assert len(cleaned) == original + twentieth_count

# Numeric
numeric, numeric_count = extract_and_count(NUMERIC, df, 'numeric')
has_dashes = numeric.str.contains(DASH)
frame = pd.DataFrame(numeric.str.split(SLASH_FORWARD).tolist(),
                    columns="month day year".split())
frame.month = clean_two_digits_isolated(frame.month)
frame.day = clean_two_digits_isolated(frame.day)
numeric_cleaned = frame_to_series(frame, numeric)

# check numeric
original = len(cleaned)
cleaned = pd.concat([cleaned, numeric_cleaned])
assert len(cleaned) == original + numeric_count
# print(len(cleaned))

# Re-combining cleaned data
cleaned_df = pd.concat([numeric_cleaned,
                       twentieth_century_cleaned,
                       words_cleaned,
                       backwards_cleaned,
                       no_day_cleaned,
                       no_day_numeric_cleaned,
                       year_only_cleaned,
                       leftovers_cleaned])
# print(cleaned_df.head())
# assert len(cleaned_df) == len(df)

# Convert to Datetime object
datetime_df = pd.to_datetime(cleaned_df, format="%m/%d/%Y")
# print(datetime_df.head())

df_sorted_dates = datetime_df.sort_values()
# print(df_sorted_dates.head())

# this process mines around 95% of the dates

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-92f89789acdf>", line 36, in <module>
    LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
NameError: name 'OR' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-pack

NameError: ignored

In [0]:

# For example if the original series was this:

#     0    1999
#     1    2010
#     2    1978
#     3    2015
#     4    1985

# The following function will return this:

#     0    2
#     1    4
#     2    0
#     3    1
#     4    3

def date_sorter():
  date_sorted = pd.Series(df_sorted_dates.index.labels[0])
  # print(date_sorted.head())
  return date_sorted

Note: This tutorial took significant inspiration from Applied Data Science in Python course from Coursera